In [41]:
from einops import *
import torch as t

# 1

In [42]:
tensor = t.arange(3, 9)
rearrange(tensor, '(b1 b2) -> b1 b2', b1=3)

tensor([[3, 4],
        [5, 6],
        [7, 8]])

In [43]:
tensor = t.arange(1, 7)
rearrange(tensor, '(b1 b2) -> b1 b2', b1=2)

tensor([[1, 2, 3],
        [4, 5, 6]])

In [44]:
tensor = t.arange(1, 7)
rearrange(tensor, 'b -> b 1 1')

tensor([[[1]],

        [[2]],

        [[3]],

        [[4]],

        [[5]],

        [[6]]])

# 2

In [48]:
temps_flat = t.Tensor([71,72,70,75,71,72,70, 68,65,60,68,60,55,59, 75,80,85,80,78,72,83])
temps = rearrange(temps_flat, '(r c) -> r c', c=7)

means = einops.reduce(temps, 'r c -> r', 'mean')
means

tensor([71.5714, 62.1429, 79.0000])

In [51]:
repeated = einops.repeat(means, 'r -> r 7')
diff_from_mean = temps - repeated

rearrange(diff_from_mean, 'r c -> (r c)')

tensor([-0.5714,  0.4286, -1.5714,  3.4286, -0.5714,  0.4286, -1.5714,  5.8571,
         2.8571, -2.1429,  5.8571, -2.1429, -7.1429, -3.1429, -4.0000,  1.0000,
         6.0000,  1.0000, -1.0000, -7.0000,  4.0000])

In [62]:
std_devs = t.std(diff_from_mean, 1)
normalized = diff_from_mean / repeat(std_devs, 'r -> r 7')
rearrange(normalized,  'r c -> (r c)')

tensor([-0.3326,  0.2494, -0.9146,  1.9954, -0.3326,  0.2494, -0.9146,  1.1839,
         0.5775, -0.4331,  1.1839, -0.4331, -1.4438, -0.6353, -0.8944,  0.2236,
         1.3416,  0.2236, -0.2236, -1.5652,  0.8944])

# 3

Compute a batched dot product. Your function should take two tensors of the same shape [i1, i2...in] (which could be any shape!) and return one where the element at index [i1, i2….in-1] is the dot product of a[i1,i2...in-1] and b[i1,i2...in-1]. For example: ( t.Tensor([[1,1,0],[0,0,1]]),t.Tensor([[1,1,0],[1,1,0]]) ) -> t.Tensor([2., 0.])

note: you can use “...” in einops (and einsum) to represent an arbitrary number of dimensions


In [80]:
def batched_dot(t1, t2):
    return t.einsum('...n,...n->...', t1, t2)

t1 = t.Tensor([[1,1,0],[0,0,1]])
t2 = t.Tensor([[1,1,0],[1,1,0]])

assert batched_dot(t1, t2).equal(t.Tensor([2., 0.]))

t3 = t.Tensor([
    [
        [1,1,0],
        [0,0,1]
    ],[
        [0,1,0],
        [1,1,1]
    ]
])
    
t4 = t.Tensor([
    [
        [1,1,0],
        [1,1,0]
    ],[
        [0,1,0],
        [1,1,1]
    ]
])
    
assert batched_dot(t3, t4).equal(t.Tensor([[2., 0.],[1., 3.]]))

# 4

In [108]:
def identity(n):  
    return 1 - (rearrange(t.arange(n**2), '(n1 n2)->n1 n2', n1=n) % (n+1)).bool().int()

identity(7)

tensor([[1, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 1]], dtype=torch.int32)

# 5

In [156]:
n =  5
p = t.Tensor([0.05, 0.1, 0.1, 0.2, 0.15, 0.4])

def rolls(n, p):
    k = p.shape[0]
    cdf = repeat(t.cumsum(p, 0), 'r-> n r', n=n)
    rolls = repeat(t.rand((n,)), 'c -> c k', k = k)

    return (rolls > cdf).sum(1)


# 6

Suppose your classifier gives you an array of scores with shape [n_inputs, n_classes], where the (i, j)-element represents how confidently the classifier thinks that example i belongs to class j. A higher number represents higher confidence, and the classifier’s prediction is the class with the highest score. 

Given such a score array and a vector of correct classes of length n_inputs, return the accuracy of the classifier: the fraction of inputs for which the maximum score corresponds to the correct class for that input.

```
(tensor([[0.75, 0.5, 0.25], [0.1, 0.5, 0.4], [0.1, 0.7, 0.2]], tensor([0, 1, 0]))
->
tensor(0.6667)
```


In [168]:
acc = lambda scores, y: (scores.argmax(1) == y).sum() / y.shape[0]


scores = t.Tensor([[0.75, 0.5, 0.25], [0.1, 0.5, 0.4], [0.1, 0.7, 0.2]])
y = t.Tensor([0, 1, 0])
acc(scores, y)

tensor(0.6667)

# 7

A shop sells K different items, charging prices[k] for item k (for k between 0 and K-1). Given a vector of item indices representing what different customers bought, calculate how much money the shop made.

```
(prices=tensor([0.5, 1, 1.5, 2, 2.5], items=tensor([0, 0, 1, 1, 4, 3, 2])))
->
tensor(9.)
```

In [183]:
sales = lambda prices, items: t.index_select(prices, 0, items).sum()

prices=t.Tensor([0.5, 1, 1.5, 2, 2.5])
items=t.Tensor([0, 0, 1, 1, 4, 3, 2]).int()
sales(prices, items)

tensor(9.)